In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import numpy as np

In [ ]:
df = pd.read_csv("/content/Placements_Dataset - Sheet1.csv")

In [ ]:
#Data Preprocessing
df.drop(columns=['sl_no'], inplace=True)

gender_encoder = LabelEncoder()
df['gender'] = gender_encoder.fit_transform(df['gender'])

ssc_scaler = MinMaxScaler()
df['ssc_p'] = ssc_scaler.fit_transform(df[['ssc_p']])


ssc_b_encoder = LabelEncoder()
df['ssc_b'] = ssc_b_encoder.fit_transform(df['ssc_b'])


hsc_scaler = MinMaxScaler()
df['hsc_p'] = hsc_scaler.fit_transform(df[['hsc_p']])


hsc_b_encoder = LabelEncoder()
df['hsc_b'] = hsc_b_encoder.fit_transform(df['hsc_b'])


hsc_s_encoder = LabelEncoder()
df['hsc_s'] = hsc_s_encoder.fit_transform(df['hsc_s'])


degree_scaler = MinMaxScaler()
df['degree_p'] = degree_scaler.fit_transform(df[['degree_p']])


degree_t_encoder = LabelEncoder()
df['degree_t'] = degree_t_encoder.fit_transform(df['degree_t'])


workex_encoder = OneHotEncoder()
workex_encoded = workex_encoder.fit_transform(df[['workex']]).toarray()
workex_df = pd.DataFrame(workex_encoded, columns=['workex_No', 'workex_Yes'])
df = pd.concat([df, workex_df], axis=1)
df.drop(columns=['workex'], inplace=True)


apt_test_scaler = MinMaxScaler()
df['apt_test_p'] = apt_test_scaler.fit_transform(df[['apt_test_p']])


specialisation_encoder = OneHotEncoder()
specialisation_encoded = specialisation_encoder.fit_transform(df[['specialisation']]).toarray()
specialisation_df = pd.DataFrame(specialisation_encoded, columns=['specialisation_Mkt&HR', 'specialisation_Mkt&Fin'])
df = pd.concat([df, specialisation_df], axis=1)
df.drop(columns=['specialisation'], inplace=True)


mba_scaler = MinMaxScaler()
df['mba_p'] = mba_scaler.fit_transform(df[['mba_p']])


status_encoder = LabelEncoder()
df['status'] = status_encoder.fit_transform(df['status'])


df.rename(columns={'status': 'placed'}, inplace=True)


cols = [col for col in df.columns if col != 'placed'] + ['placed']
df = df[cols]
salary_bands = [(0, 250000), (250000, 350000), (350000, 450000), (450000, 550000), (550000, float('inf'))]
labels = [1, 2, 3, 4, 5]

def map_salary_to_band(salary):
    for i, (lower, upper) in enumerate(salary_bands):
        if salary >= lower and salary < upper:
            return labels[i]
    return labels[-1]

df['salary_band'] = df['salary'].apply(map_salary_to_band)

df.loc[df['placed'] == 0, 'salary'] = np.nan
df.loc[df['placed'] == 0, 'salary_band'] = np.nan
df.drop(columns=['salary'], inplace=True)
df['salary_band'].fillna(0, inplace=True)

In [ ]:
df.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,apt_test_p,mba_p,workex_No,workex_Yes,specialisation_Mkt&HR,specialisation_Mkt&Fin,placed,salary_band
0,1,0.538240,1,0.889621,1,1,0.195122,2,0.104167,0.284483,1.0,0.0,0.0,1.0,1,2.0
1,1,0.792414,0,0.680890,1,2,0.670244,2,0.760417,0.564843,0.0,1.0,1.0,0.0,1,1.0
2,1,0.497011,0,0.510708,0,0,0.341463,0,0.520833,0.247001,1.0,0.0,1.0,0.0,1,2.0
3,1,0.311482,0,0.247117,0,2,0.048780,2,0.333333,0.308096,1.0,0.0,0.0,1.0,0,0.0
4,1,0.925788,0,0.602965,0,1,0.568293,0,0.975000,0.160795,1.0,0.0,1.0,0.0,1,3.0


In [ ]:
correlation_with_placed = df.corr()['placed']
print(correlation_with_placed)

gender                    0.090670
ssc_p                     0.607889
ssc_b                     0.037297
hsc_p                     0.491228
hsc_b                     0.016945
hsc_s                     0.033442
degree_p                  0.479861
degree_t                 -0.020352
apt_test_p                0.127639
mba_p                     0.076922
workex_No                -0.276060
workex_Yes                0.276060
specialisation_Mkt&HR     0.250655
specialisation_Mkt&Fin   -0.250655
placed                    1.000000
salary_band               0.790013
Name: placed, dtype: float64


In [ ]:
selected_columns = df.columns.difference(['placed'])
correlation_with_salary = df[selected_columns].corrwith(df['salary_band'])
print(correlation_with_salary)

apt_test_p                0.169656
degree_p                  0.388310
degree_t                  0.100417
gender                    0.166462
hsc_b                     0.034632
hsc_p                     0.421611
hsc_s                     0.100293
mba_p                     0.147027
salary_band               1.000000
specialisation_Mkt&Fin   -0.245613
specialisation_Mkt&HR     0.245613
ssc_b                     0.031572
ssc_p                     0.507075
workex_No                -0.286049
workex_Yes                0.286049
dtype: float64


In [ ]:
X_placement = df.drop(columns=['placed', 'salary_band'])
y_placement = df['placed']

X_salary = df.drop(columns=['placed'])
y_salary = df['salary_band']


In [ ]:
from sklearn.model_selection import train_test_split

X_train_placement, X_test_placement, y_train_placement, y_test_placement = train_test_split(X_placement, y_placement, test_size=0.2, random_state=42)
X_train_salary, X_test_salary, y_train_salary, y_test_salary = train_test_split(X_salary, y_salary, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
def create_mlp(input_shape, num_classes):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

In [20]:
# Initialize the model
input_shape = X_train_placement.shape[1]
num_classes = len(np.unique(y_train_placement))  # Number of unique classes
model = create_mlp(input_shape, num_classes)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [21]:
# Define callbacks (e.g., early stopping)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
history = model.fit(X_train_placement, y_train_placement,
                    validation_data=(X_val_placement, y_val_placement),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping],
                    verbose=1)


NameError: name 'X_val_placement' is not defined